In [1]:
import csv
import re
from collections import defaultdict

f = open("tweets.txt", "r")
text = f.read()
#text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

In [2]:
clean_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())

In [3]:
clean_text
with open('clean_text.txt', 'w') as f:
    f.write(clean_text)

In [13]:
from nltk.corpus import stopwords 
#remove stopwords
use_stopwords = stopwords.words('english')
token_li = " ".join([i for i in clean_text.split() if i not in use_stopwords])
token_li = token_li.split()
token_li.remove('https')    
print(token_li)

['savage', 'people', 'survived', 'acting', 'like', 'see', 'thing', 'kind', 'like', 'working', 'Harvey', 'Weinstein', 'HarveyWeinstein', 'goldenglobes2020', 'MeToo', 'NoMeansNo', 'left', 'gem', 'list', 'Truth', 'Power', 'category', 'Ricky', 'Gervais', 'movie', 'called', 'Birdbox', 'movie', 'people', 'pretend', 'like', 'see', 'anything', 'Kinda', 'like', 'working', 'Harvey', 'Weinstein', 'audience', 'gasp', 'More', 'two', 'years', 'wave', 'sexual', 'assault', 'allegations', 'Harvey', 'Weinstein', 'ushered', 'MeToo', 'movement', 'disgraced', 'movie', 'mogul', 'faces', 'criminal', 'trial', 'could', 'put', 'behind', 'bars', 'rest', 'life', 'More', 'two', 'years', 'wave', 'sexual', 'assault', 'allegations', 'Harvey', 'Weinstein', 'ushered', 'MeToo', 'movement', 'disgraced', 'movie', 'mogul', 'faces', 'another', 'reckoning', 'criminal', 'trial', 'could', 'put', 'behind', 'bars', 'rest', 'life', 'More', 'two', 'years', 'wave', 'sexual', 'assault', 'allegations', 'Harvey', 'Weinstein', 'ushered

In [39]:
import codecs
from nltk.tokenize import word_tokenize
from collections import defaultdict

person_counter = defaultdict(int)  
person_per_line = []
relationships = {}

In [40]:
with open('tweets_cleaned.csv',encoding="utf8", errors='ignore') as csvfile:
    readCSV = csv.reader(csvfile,delimiter=',')
    
    tweets = []
    for row in readCSV:
        tweets.append(row[10])

In [ ]:

# divide the text into paragraghs
for line in tweets:
    # tokenize each paragragh
    poss = word_tokenize(line)
    # for each paragragh, we append a list in the list: person_per_line. 
    #The list will record the characters appearing in the current paragragh. 
    person_per_line.append([])
    # if a word is not a character name, then continue;
    # else, store the name in the last list in person_per_line.
    for w in poss:
        if w not in token_li:
            continue
        person_per_line[-1].append(w)
    #if the word is not in the person_counter, which means the relationship start with this name does not exist,then add the name to the relationship dictionary.
        if person_counter.get(w) is None: #Python 字典(Dictionary) get() 函数返回指定键的值，如果值不在字典中返回默认值None
            relationships[w] = {}
    # else, the key of person_counter[w] + 1
        person_counter[w] +=1


In [ ]:
print ("start to process edges")
#遍历每一段，笛卡尔积形式统计人物关系
# p is a list in person_per_line, if name1 and name 2 both in p, and name1 != name2, then add a relationship to the dictionary. 
for p in person_per_line:
    for name1 in p:
        for name2 in p:
            if name1 == name2:
                continue
            if relationships[name1].get(name2) is None:
                relationships[name1][name2] = 1
            else:
                relationships[name1][name2] +=1

In [37]:
# for nodes.csv, name in person_counter is Id and Label, and the frequency of each character is Weight.
with codecs.open("node.csv","a+","utf-8") as f:
    f.write("Id,Label,Weight\r\n")
    for name, times in person_counter.items():
        f.write(name+","+name+","+str(times)+"\r\n")
# for edge.csv, name in relationships{} is         
with codecs.open("edge.csv","a+","utf-8") as f:
    f.write("Source,Target,Weight\r\n")
    # name is the key(Source), and edges is the value of name. edges is also a dictionary, which stores the target names and the weight of each target.
    for name, edges in relationships.items():
        # v is the target name, w is the weight.
        for v,w in edges.items():
            #
            if w >3:
                f.write(name+","+ v + "," + str(w) + "\r\n")